<a href="https://colab.research.google.com/github/slesarev-hub/Column-store-compression-memory-usage/blob/master/compression_memory_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set
import csv
import copy
import sys
import scipy
from IPython import display
from numpy import savez_compressed
import os
import gzip

In [0]:
!git clone https://github.com/slesarev-hub/ssb-dbgen
%cd ssb-dbgen
!ls
!cmake . && cmake --build .

In [0]:
# -s - scale factor
!./dbgen -v -s 2

[Super-Scalar RAM-CPU Cache Compression](https://paperhub.s3.amazonaws.com/7558905a56f370848a04fa349dd8bb9d.pdf)

**Prefix Suppression (PS)** - for numeric data if actual values tend to be significantly smaller than the largest value of the type 

**Frame Of Reference (FOR)** - for each disk block
the minimum $min_C$ value for the numeric column C, and
then stores all column values $c[i]$ as $c[i] − min_C$ in an integer of only $log_2(max_C −min_C +1)$ bits

**Dictionary Compression**

**Delta Compression**

**Patched Frame-of-Reference (PFOR)** - the small integers are
positive offsets from a base value. One (possibly negative) base value is used per disk block, values below base stored as exceptions

**PFOR-DELTA** - decompression consists of PFOR-decompression, and then computing the running sum on the result.

**PDICT** - integer codes
refer to a position in an array of values

[Simple Solutions
for Compressed Execution
in Vectorized Database System](https://homepages.cwi.nl/~boncz/msc/2011-AlaLuszczak.pdf)

[The Implementation and Performanceof Compressed Databases](https://www.researchgate.net/publication/2765968_The_Implementation_and_Performance_of_Compressed_Databases)

[Integrating Compression and Execution in
Column-Oriented Database Systems](http://db.csail.mit.edu/projects/cstore/abadisigmod06.pdf)

------------
[Query Optimization In Compressed Database Systems](http://www.cs.cornell.edu/zhychen/paper/sigmod01-camera.pdf)

----------

Handbook of Data Compression

David Salomon
With Contributions by David Bryant
Giovanni Motta

[huffman+LZ77](https://tools.ietf.org/html/rfc1951)


In [0]:
def align_to_byte(n_bits):
  return (n_bits + 7) // 8

def byte_size_of(number):
  return align_to_byte(len(format(number, 'b')))

In [0]:
def for_volume(raw_value_size, column):
  """Super-Scalar RAM-CPU Cache Compression
  minimum BY BLOCK + [offsets from minimum]"""
  cmprs_value_size = byte_size_of(np.max(column) - np.min(column))
  # 1 byte to encode compressed value size, one value is minimum, so len = len - 1
  return raw_value_size + 1 + cmprs_value_size * (len(column) - 1)

In [0]:
def ps_volume(raw_value_size, column):
  """Super-Scalar RAM-CPU Cache Compression
  remove common prefix"""
  str_column = column.astype(str)
  m = np.min([len(s) for s in str_column])
  low, high = 0, m
  prefix = ""
  while low <= high:
    mid = low + (high - low)//2
    if np.all([s[low:mid+1] == str_column[0][low:mid+1] for s in str_column]):
      prefix += str_column[0][low:mid+1]
      low = mid + 1
    else:
      high = mid - 1
  if len(prefix) == 0 and isinstance(column[0], str):
    compressed_volume = len(column[0])*len(column)
  elif len(prefix) == 0:
    compressed_volume = byte_size_of(np.max(column))*len(column)
  elif isinstance(column[0], str):
    compressed_volume = 1 + (len(column[0])-len(prefix))*len(column) #sizeof(char) = 1
  else:
    compressed_volume = 1 + byte_size_of(int(str(np.max(column))[len(prefix):]))*len(column)
  return compressed_volume

In [0]:
def delta_volume(raw_value_size, column):
  """Super-Scalar RAM-CPU Cache Compression"""
  max_delta = max(column[i] - column[i - 1] for i in range(1, len(column)))
  # delta can be negative, while the absolute value are of interest for us
  min_delta = min(column[i] - column[i - 1] for i in range(1, len(column)))
  # as in for_volumn, but we hate to store min_delta together with the column[0] elemnt
  # therefor, 2 * raw_value_size
  return 2 * raw_value_size + 1 + byte_size_of(max_delta - min_delta) * (len(column) - 1)

In [0]:
def dict_volume(raw_value_size, column):
  """Super-Scalar RAM-CPU Cache Compression"""
  dict_values = np.unique(column)
  # compute dictionary volume
  # dictionary is a structure with fast "get value by key"
  # assume that it is array _keys_: i --> offset of i-th value and a memory buffer with values
  # then value with key _i_ is in _buffer + keys[i]_ 
  # assume that offset is 32bit (4byte) len. Sometimes it can be significantly smaller,
  # you can adjust the formula if you want
  # for an array of integers, we can store actual values instead of offsets
  offset_size = 4
  if not isinstance(column[0], np.int64):
    dict_size = sum(len(c) for c in dict_values) + len(dict_values) * offset_size 
  else:
    dict_size = len(dict_values) * byte_size_of(np.max(dict_values))
  dict_key_size = byte_size_of(len(dict_values))
  return len(column) * dict_key_size + dict_size

In [0]:
def pfor_analyze_bits(column, b):
  lo = 0
  l = 0
  m = 0
  rng = 1<<b
  for hi in range(len(column)):
    if column[hi] - column[lo] >= rng:
      if hi - lo > l:
        m = lo
        l = hi - lo
      while(column[hi] - column[lo] >= rng):
        lo += 1
  if len(column) - 1 - lo > l:
    m = lo
    l = hi - lo
  return (m,l+1)

def pfor_volume(raw_value_size, column):
  """Super-Scalar RAM-CPU Cache Compression
  compress [base_min, base_max] into 24 bit values, other values stay uncompressed"""
  unique = np.unique(column)
  s = len(column)
  b_ans = np.inf
  b = 1
  sorted_column = np.sort(column)
  base = 0
  for b_curr in range(1,8*raw_value_size):
    base_idx, l = pfor_analyze_bits(sorted_column, b_curr)
    new_b_ans = b_curr + (s - l)*8*raw_value_size/s
    if new_b_ans < b_ans:
      base = sorted_column[base_idx]
      b_ans = new_b_ans
      b = b_curr
  not_compressed = [val for val in column if val < base or val > base + (1 << (b + 1)) - 1]
  if len(not_compressed) == 0:
    not_compressed_volume = 0
  else:
    not_compressed_volume = len(not_compressed)*raw_value_size
  entry_point_volume = 32*(s//128)
  compressed_volume = not_compressed_volume + align_to_byte((s - len(not_compressed))*b) + entry_point_volume
  return compressed_volume

In [0]:
def pfor_delta_volume(raw_value_size, column):
  """Super-Scalar RAM-CPU Cache Compression"""
  d_column = [column[i]-column[i-1] for i in range(1,len(column))]
  d_column.insert(0, column[0])
  return pfor_volume(raw_value_size, np.array(d_column))

In [0]:
def pdict_volume(raw_value_size, column):
  """Super-Scalar RAM-CPU Cache Compression"""
  if isinstance(column[0], str):
    raw_value_size = int(np.round(np.mean([len(s) for s in column])))
  col = copy.deepcopy(column)
  unique, counts = np.unique(column, return_counts=True)
  order = np.stack((counts, unique), axis=-1)
  order = order[order[:,0].argsort()[::-1]]
  s = len(column)
  b_ans = np.inf;
  b = 1
  for b_curr in range(1,8*raw_value_size):
    new_b_ans = b_curr + (1-np.sum(counts[:2**b_curr]))*8*raw_value_size
    if new_b_ans < b_ans:
      b_ans = new_b_ans
      b = b_curr
  entry_point_volume = 32*(s//128)
  compressed_volume = raw_value_size*np.sum(counts[2**b:]) + align_to_byte(b*np.sum(counts[:2**b])) + entry_point_volume 
  return compressed_volume

In [0]:
def array2string(a):
  s = ""
  for c in a:
    s += str(c) + '\n'
  return s

def get_uncompressed_volume(raw_value_size, column):
  return raw_value_size*len(column)

In [0]:
def bit_vector_volume(raw_value_size, column):
  """Integrating Compression and Execution in Column-Oriented Database Systems"""
  return len(np.unique(column))*(byte_size_of(np.max(column)) + byte_size_of(len(column)))

def bit_dict_volume(raw_value_size, column):
  """Integrating Compression and Execution in Column-Oriented Database Systems"""
  unique = np.unique(column)
  val_bits = len(format(len(unique), 'b'))
  dict_val_size = [8, 16, 24, 32, 40]
  dict_val_size = [size for size in dict_val_size if size >= val_bits]
  compressed_cnt = [bts//val_bits for bts in dict_val_size]
  dict_size = []
  for i in range(len(dict_val_size)):
    cnt = compressed_cnt[i]
    dict_vals = set()
    pos = 0
    while pos + cnt < len(column):
      dict_vals.add(array2string(column[pos:pos+cnt]))
      pos += cnt
    dict_size.append(len(dict_vals)*align_to_byte(dict_val_size[i]))
  compressed_volume = [int(dict_size[i] + (len(column)//compressed_cnt[i] + 1)*align_to_byte(dict_val_size[i])) for i in range(len(dict_val_size))]
  return min(compressed_volume)

In [0]:
def rle(raw_value_size, column):
  """Simple Solutions for Compressed Execution in Vectorized Database System
  apply bwt to column as singlw string, then apply rle to each character
  also we should store dict for bwt string characters, because of special symbols in bwt"""
  if isinstance(column[0], str):
    raw_value_size = int(np.round(np.mean([len(s) for s in column])))
  unique = list(set(bwt(column)))
  encoded_value_size = byte_size_of(len(unique))
  compressed_volume = len(unique)*(encoded_value_size + raw_value_size)
  cnt = 1
  prev = ""
  freq = []
  for c in column:
    if c != prev:
      if prev != "":
        freq.append((prev, cnt))
      cnt = 1
      prev = c
    else:
      cnt += 1
  else:
    freq.append((c, cnt))
  max_cnt = 0
  cnt_of_sequences = 0
  for val, cnt in freq:
    compressed_volume += encoded_value_size
    if cnt != 1:
      cnt_of_sequences += 1
      max_cnt = max(cnt, max_cnt)
  compressed_volume += cnt_of_sequences*byte_size_of(max_cnt)
  return compressed_volume

def bwt(column):
  str_col = array2string(column)
  #assert ":" not in str_col and ";" not in str_col
  #str_col = ":" + str_col + ";"  
  table = sorted(str_col[i:] + str_col[:i] for i in range(len(str_col)))  
  last_column = [row[-1:] for row in table]  
  return "".join(last_column)  

In [0]:
def row_huffman_volume(raw_value_size, column):
  """no interim aligning to int bytes; compress each row into code value"""
  if isinstance(column[0], str):
    raw_value_size = int(np.round(np.mean([len(s) for s in column])))
  col = [str(c) for c in column]
  unique, cnt = np.unique(col, return_counts=True)
  l = len(cnt)
  counts = [[set([unique[i]]), cnt[i]] for i in range(l)]
  row_dict = {val : 0 for val in unique}
  counts.sort(key=lambda x : x[1], reverse=True)
  while len(counts) > 1:
    union_cnt = counts[0][1] + counts[1][1]
    union_vals = counts[0][0].union(counts[1][0])
    counts.pop(0)
    counts.pop(0)
    counts.append([union_vals,union_cnt])
    for u in union_vals:
      row_dict[u] += 1
  compressed_volume = 0
  for c in col:
    compressed_volume += row_dict[c]
  meta_info_volume = raw_value_size*len(unique) + max(list(row_dict.values()))*len(row_dict)
  return align_to_byte(compressed_volume) + meta_info_volume

In [0]:
def huffman_volume(raw_value_size, column, symbols_to_code=3):
  """no interim aligning to int bytes; compress each 'symbols_to_code' symbols into code value"""
  col_concat = ""
  for c in column:
    col_concat += str(c) + '\n'
  col = []
  pos = symbols_to_code
  while pos <= len(column):
    col.append(col_concat[pos-symbols_to_code:pos])
    pos += symbols_to_code
  #assume that values in row_huffman_volume is strings of symbols_to_code chars
  compressed_volume = row_huffman_volume(symbols_to_code, col) 
  #save rest as uncompressed 1-byte chars
  if pos != len(column):
    compressed_volume += len(column) - pos + symbols_to_code
  return compressed_volume

In [0]:
def lz78(raw_value_size, column):
  """aligning to bytes taken into account"""
  col_concat = ""
  for c in column:
    col_concat += str(c) + '\n'
  #format of dict: (dict_id, char), size fo record: size_of_dict_id + 1
  records = set()
  tokens_cnt = 0
  s = 0
  for i in range(1,len(col_concat)):
    if col_concat[s:i] not in records:
      records.add(col_concat[s:i])
      s = i
      tokens_cnt += 1
  compressed_volume = tokens_cnt*(byte_size_of(len(records)) + 1)
  return compressed_volume 

In [0]:
def page_level_compression(algo, page_size, raw_value_size, column):
  """divide column into parts of page_size and apply algo, pages_size in bytes"""
  compressed_volume = 0
  values_in_page = page_size//raw_value_size
  l = len(column)
  full_pages_cnt = l//values_in_page
  bound = 0
  for i in range(full_pages_cnt):
    compressed_volume += algo(raw_value_size, column[bound:bound+values_in_page])
    bound += values_in_page
  if len(column[bound:]) != 0:
    compressed_volume += algo(raw_value_size, column[bound:])
  return compressed_volume

In [0]:
def np_columns_plots(columns, sizes_scale, labels, page_size, algos):
  c = len(columns)
  s = sizes_scale
  fig, axarr = plt.subplots(c//s, s, figsize=(30,20))
  fig.subplots_adjust(hspace=.5, wspace=.5)
  if 'без сжатия' not in labels:
    labels.append('без сжатия')
  else:
    print('столбец \"без сжатия добавляется\" автоматически')
    sys.exit()
  for i in range(c//s):
    for j in range(s):
      column, raw_value_size = columns[s*i+j]
      origin = raw_value_size*len(column)
      sizes = [page_level_compression(f, page_size, raw_value_size, column) for f in algos]
      sizes.append(origin)
      print(sizes)
      x = np.arange(len(labels))
      axarr[i,j].set_xticks(x)
      axarr[i,j].set_xticklabels(labels)
      axarr[i,j].set_ylim(10,3*np.max(sizes)/2)
      axarr[i,j].bar(x, sizes)
      axarr[i,j].set_title('range: '+str(np.min(column))+" - "+str(np.max(column))+"\ncolumn size: "+str(len(column)), y=1.)   

In [0]:
def get_plots(page_size, columns):
  fig, axarr = plt.subplots(1, len(columns), figsize=(40,5))
  fig.subplots_adjust(hspace=0.7, wspace=0.3)
  for i in range(len(columns)):
    c,type_size = columns[i]
    labels = ['ps', 'for', 'pfor', 'delta', 'pfor_delta', 'dict', 'pdict', 'bit-vec', 'bit-dict', 'rle', 'origin']
    x = np.arange(len(labels))
    name = c.name
    c = c.values
    algos = [ps_volume, for_volume, pfor_volume, delta_volume, pfor_delta_volume, dict_volume, pdict_volume, bit_vector_volume, bit_dict_volume, rle]
    sizes = [page_level_compression(f, page_size, type_size, c) for f in algos]
    print(sizes)
    sizes.append(type_size*len(c))
    axarr[i].set_xticks(x)
    axarr[i].set_xticklabels(labels)
    axarr[i].set_ylim(10,3*np.max(sizes)/2)
    axarr[i].bar(x, sizes)   
    axarr[i].set_title('range: '+str(np.min(c))+" - "+str(np.max(c))+"\ncolumn name: "+name, y=1.2)

In [0]:
def get_hists(columns):
  fig, axarr = plt.subplots(1, len(columns), figsize=(30,5))
  fig.subplots_adjust(hspace=0.7, wspace=0.5)
  for i in range(len(columns)):
    axarr[i].hist(columns[i], bins=20, density=True)
    axarr[i].set_title(columns[i].name)

In [0]:
def npz_volume(column):
  savez_compressed('tmp.npz', column)
  size = os.path.getsize('tmp.npz')
  os.remove('tmp.npz')
  return size

In [0]:
def gzip_volume(column):
  with gzip.open("tmp.gz", "wb") as f:
	  f.write(column)
  size = os.path.getsize('tmp.gz')
  os.remove('tmp.gz')
  return size

In [0]:
def create_table(columns, algos, labels, res):
  for i in range(len(columns)):
    res[i].append(npz_volume(columns[i].values))
    res[i].append(gzip_volume(columns[i].values))
  df = pd.DataFrame(res, index = [c.name for c in columns], columns = labels)
  compression_rates = []
  for c in df:
    column_rates = []
    for i in range(len(df[c])):
      column_rates.append(df[c][i]/df['origin'][i])
    compression_rates.append(np.mean(column_rates))
  compression_rates = pd.DataFrame({'rate' : compression_rates},index = labels).T
  return df, compression_rates

In [0]:
def numeric_columns_compression_volumes_df(columns, raw_value_sizes, page_size):
  algos = [ps_volume, for_volume, pfor_volume, delta_volume, pfor_delta_volume, dict_volume, pdict_volume, bit_vector_volume, bit_dict_volume, rle, row_huffman_volume, huffman_volume, lz78, get_uncompressed_volume]
  labels = ['ps', 'for', 'pfor', 'delta', 'pfor_delta', 'dict', 'pdict', 'bit-vec', 'bit-dict', 'rle', 'row_huff', 'huff', 'lz78', 'origin', 'npz', 'gzip']
  res = [[page_level_compression(f, page_size, raw_value_sizes[i], columns[i].values) for f in algos] for i in range(len(columns))]
  return create_table(columns, algos, labels, res)

In [0]:
def columns_compression_volumes_df(columns, algos, labels, raw_value_sizes, page_size):
  res = [[page_level_compression(f, page_size, raw_value_sizes[i], columns[i].values) for f in algos] for i in range(len(columns))]
  return create_table(columns, algos, labels, res)

In [0]:
def display_pair(pds):
  display.display(pds[0])
  print('\n')
  display.display(pds[1])

**light-weight** on SSB Date (scale factor 2, 224KB), uniformly distributed, columns sizes less than page size

In [28]:
column_names = ['D_DATEKEY','D_DATE','D_DAYOFWEEK','D_MONTH','D_YEAR','D_YEARMONTHNUM','D_YEARMONTH','D_DAYNUMINWEEK','D_DAYNUMINMONTH',
                'D_DAYNUMINYEAR','D_MONTHNUMINYEAR','D_WEEKNUMINYEAR','D_SELLINGSEASON','D_LASTDAYINWEEKFL','D_LASTDAYINMONTHFL','D_HOLIDAYFL','D_WEEKDAYFL','']
date_tbl = pd.read_csv('/content/ssb-dbgen/date.tbl', sep='|\n', delimiter='|', header=None, names=column_names)
date_tbl = date_tbl.drop(columns='')
print(date_tbl.info())
date_tbl

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 17 columns):
D_DATEKEY             2557 non-null int64
D_DATE                2557 non-null object
D_DAYOFWEEK           2557 non-null object
D_MONTH               2557 non-null object
D_YEAR                2557 non-null int64
D_YEARMONTHNUM        2557 non-null int64
D_YEARMONTH           2557 non-null object
D_DAYNUMINWEEK        2557 non-null int64
D_DAYNUMINMONTH       2557 non-null int64
D_DAYNUMINYEAR        2557 non-null int64
D_MONTHNUMINYEAR      2557 non-null int64
D_WEEKNUMINYEAR       2557 non-null int64
D_SELLINGSEASON       2557 non-null object
D_LASTDAYINWEEKFL     2557 non-null int64
D_LASTDAYINMONTHFL    2557 non-null int64
D_HOLIDAYFL           2557 non-null int64
D_WEEKDAYFL           2557 non-null int64
dtypes: int64(12), object(5)
memory usage: 339.7+ KB
None


,D_DATEKEY,D_DATE,D_DAYOFWEEK,D_MONTH,D_YEAR,D_YEARMONTHNUM,D_YEARMONTH,D_DAYNUMINWEEK,D_DAYNUMINMONTH,D_DAYNUMINYEAR,D_MONTHNUMINYEAR,D_WEEKNUMINYEAR,D_SELLINGSEASON,D_LASTDAYINWEEKFL,D_LASTDAYINMONTHFL,D_HOLIDAYFL,D_WEEKDAYFL
0,19920101,"January 1, 1992",Thursday,January,1992,199201,Jan1992,5,1,1,1,1,Winter,0,0,1,1
1,19920102,"January 2, 1992",Friday,January,1992,199201,Jan1992,6,2,2,1,1,Winter,0,0,0,1
2,19920103,"January 3, 1992",Saturday,January,1992,199201,Jan1992,7,3,3,1,1,Winter,1,0,0,0
3,19920104,"January 4, 1992",Sunday,January,1992,199201,Jan1992,1,4,4,1,1,Winter,0,0,0,0
4,19920105,"January 5, 1992",Monday,January,1992,199201,Jan1992,2,5,5,1,1,Winter,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2552,19981227,"December 27, 1998",Monday,December,1998,199812,Dec1998,2,27,361,12,52,Christmas,0,0,0,1
2553,19981228,"December 28, 1998",Tuesday,December,1998,199812,Dec1998,3,28,362,12,52,Christmas,0,0,0,1
2554,19981229,"December 29, 1998",Wednesday,December,1998,199812,Dec1998,4,29,363,12,52,Christmas,0,0,0,1
2555,19981230,"December 30, 1998",Thursday,December,1998,199812,Dec1998,5,30,364,12,53,Christmas,0,0,0,1


In [29]:
for i in range(1,5):
  print(huffman_volume(4,date_tbl.D_YEAR.values, i))

852
507
369
341


In [30]:
display_pair(numeric_columns_compression_volumes_df([date_tbl.D_YEAR, date_tbl.D_DATEKEY, date_tbl.D_YEARMONTHNUM, date_tbl.D_DAYNUMINYEAR],[4,4,4,4],16384))

,ps,for,pfor,delta,pfor_delta,dict,pdict,bit-vec,bit-dict,rle,row_huff,huff,lz78,origin,npz,gzip
D_YEAR,2558,2561,1567,2565,932,2571,1567,28,993,71,963,369,2604,10228,292,87
D_DATEKEY,7672,5117,5722,5121,1254,15342,4444,15342,7671,2612,44556,994,10164,10228,4180,3957
D_YEARMONTHNUM,5115,5117,3805,2565,955,2809,2846,420,2642,223,2997,960,7293,10228,437,238
D_DAYNUMINYEAR,5114,5117,3485,5121,951,5846,3485,1464,4876,2612,7508,3614,7197,10228,1048,776


,ps,for,pfor,delta,pfor_delta,dict,pdict,bit-vec,bit-dict,rle,row_huff,huff,lz78,origin,npz,gzip
rate,0.500073,0.437818,0.35635,0.375733,0.10002,0.649394,0.301672,0.421734,0.395532,0.134875,1.369378,0.145116,0.666259,1.0,0.145605,0.123631


In [0]:
def page_dict_vs_full_df(columns, raw_value_sizes, page_size):
  labels = ['dict', 'pdict', 'bit-vec', 'bit-dict', 'row_huff', 'huff', 'lz78', 'origin']
  algos = [dict_volume, pdict_volume, bit_vector_volume, bit_dict_volume, row_huffman_volume, huffman_volume, lz78, get_uncompressed_volume]
  page_level_sizes = [[page_level_compression(f, page_size, raw_value_sizes[i], columns[i].values) for f in algos] for i in range(len(columns))]
  column_level_sizes = [[f(raw_value_sizes[i], columns[i].values) for f in algos] for i in range(len(columns))]
  page_level_sizes_df = pd.DataFrame(page_level_sizes, index = [c.name for c in columns], columns = labels)
  column_level_sizes_df = pd.DataFrame(column_level_sizes, index = [c.name for c in columns], columns = labels)
  return (page_level_sizes_df, column_level_sizes_df)

In [32]:
pds = page_dict_vs_full_df([date_tbl.D_YEAR, date_tbl.D_DATEKEY, date_tbl.D_YEARMONTHNUM, date_tbl.D_DAYNUMINYEAR], [4,4,4,4], 16384)
print('page level')
display.display(pds[0])
print('\ncolumn level')
display.display(pds[1])

page level


,dict,pdict,bit-vec,bit-dict,row_huff,huff,lz78,origin
D_YEAR,2571,1567,28,993,963,369,2604,10228
D_DATEKEY,15342,4444,15342,7671,44556,994,10164,10228
D_YEARMONTHNUM,2809,2846,420,2642,2997,960,7293,10228
D_DAYNUMINYEAR,5846,3485,1464,4876,7508,3614,7197,10228



column level


,dict,pdict,bit-vec,bit-dict,row_huff,huff,lz78,origin
D_YEAR,2571,1567,28,993,963,369,2604,10228
D_DATEKEY,15342,4444,15342,7671,44556,994,10164,10228
D_YEARMONTHNUM,2809,2846,420,2642,2997,960,7293,10228
D_DAYNUMINYEAR,5846,3485,1464,4876,7508,3614,7197,10228


SSB Lineorder (scale factor 2, 1.2GB)

In [33]:
column_names = ['lo_orderkey', 'lo_linenumber', 'lo_custkey', 'lo_partkey', 'lo_suppkey', 'lo_orderdate', 'lo_orderpriority', 'lo_shippriority', 'lo_quantity', 'lo_extendedprice',
                'lo_ordertotalprice', 'lo_discount', 'lo_revenue', 'lo_supplycost', 'lo_tax', 'lo_commitdate', 'lo_shipmode','']
lineorder_tbl = pd.read_csv('/content/ssb-dbgen/lineorder.tbl', sep='|\n', delimiter='|', header=None, names=column_names)
lineorder_tbl = lineorder_tbl.drop(columns='')[:1998049]
print(lineorder_tbl.info())
lineorder_tbl

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998049 entries, 0 to 1998048
Data columns (total 17 columns):
lo_orderkey           int64
lo_linenumber         int64
lo_custkey            int64
lo_partkey            int64
lo_suppkey            int64
lo_orderdate          int64
lo_orderpriority      object
lo_shippriority       int64
lo_quantity           int64
lo_extendedprice      int64
lo_ordertotalprice    int64
lo_discount           int64
lo_revenue            int64
lo_supplycost         int64
lo_tax                int64
lo_commitdate         int64
lo_shipmode           object
dtypes: int64(15), object(2)
memory usage: 259.1+ MB
None


,lo_orderkey,lo_linenumber,lo_custkey,lo_partkey,lo_suppkey,lo_orderdate,lo_orderpriority,lo_shippriority,lo_quantity,lo_extendedprice,lo_ordertotalprice,lo_discount,lo_revenue,lo_supplycost,lo_tax,lo_commitdate,lo_shipmode
0,1,1,36473,155190,1655,19960130,2-HIGH,0,17,2116823,10523209,4,2032150,74711,2,19960311,TRUCK
1,1,2,36473,67310,326,19960130,2-HIGH,0,36,4598316,10523209,9,4184467,76638,6,19960327,MAIL
2,1,3,36473,63700,142,19960130,2-HIGH,0,8,1330960,10523209,10,1197864,99822,2,19960402,REG AIR
3,1,4,36473,2132,1885,19960130,2-HIGH,0,28,2895564,10523209,9,2634963,62047,6,19960427,AIR
4,2,1,41224,106170,2132,19960817,1-URGENT,0,38,4469446,21179418,0,4469446,70570,5,19960930,RAIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998044,1997507,4,41302,190440,413,19971117,5-LOW,0,10,1530440,25961525,8,1408004,91826,1,19971222,TRUCK
1998045,1997507,5,41302,114750,296,19971117,5-LOW,0,4,705900,25961525,9,642369,105885,3,19971225,AIR
1998046,1997507,6,41302,198272,3049,19971117,5-LOW,0,4,548108,25961525,6,515221,82216,8,19980210,MAIL
1998047,1997507,7,41302,152451,3841,19971117,5-LOW,0,19,2856555,25961525,4,2742292,90207,4,19971226,MAIL


In [34]:
display_pair(numeric_columns_compression_volumes_df([lineorder_tbl.lo_ordertotalprice, lineorder_tbl.lo_discount, lineorder_tbl.lo_revenue, lineorder_tbl.lo_supplycost, lineorder_tbl.lo_tax, lineorder_tbl.lo_commitdate], [4,4,4,4,4,4], 16384))

,ps,for,pfor,delta,pfor_delta,dict,pdict,bit-vec,bit-dict,rle,row_huff,huff,lz78,origin,npz,gzip
lo_ordertotalprice,7992196,7992684,6851255,7994636,2685958,5995054,3119418,2998434,4925295,954610,9747474,2774656,12123756,7992196,2508735,2502457
lo_discount,1998049,2000001,1498513,2001953,1748269,2003417,1498513,16104,1058560,2008807,933485,1126037,2283963,7992196,1514072,1404229
lo_revenue,5994147,5995123,6243958,7990541,6495111,9987305,3496562,9985345,6084369,2024889,34950078,5821065,11608773,7992196,7687222,7632904
lo_supplycost,5994147,5995123,4520880,5997075,4745478,9043211,3496562,8411855,5994117,2024889,29875653,4959055,8891088,7992196,5497424,5359103
lo_tax,1998049,2000001,1498513,2001953,1533536,2002441,1498513,13176,1039040,1997903,843244,1008433,2135478,7992196,1410552,1288684
lo_commitdate,5994635,3997562,4495586,5997075,4745078,7816230,3246806,5730198,5906685,2024605,17067479,1106652,7466028,7992196,3999523,3903342


,ps,for,pfor,delta,pfor_delta,dict,pdict,bit-vec,bit-dict,rle,row_huff,huff,lz78,origin,npz,gzip
rate,0.62501,0.583496,0.523609,0.666968,0.45781,0.768409,0.341091,0.566284,0.52151,0.230135,1.948096,0.350256,0.928178,1.0,0.471659,0.460673


In [35]:
pds = page_dict_vs_full_df([lineorder_tbl.lo_ordertotalprice],[4],16384)
print('page level')
display.display(pds[0])
print('\ncolumn level')
display.display(pds[1])

page level


,dict,pdict,bit-vec,bit-dict,row_huff,huff,lz78,origin
lo_ordertotalprice,5995054,3119418,2998434,4925295,9747474,2774656,12123756,7992196



column level


,dict,pdict,bit-vec,bit-dict,row_huff,huff,lz78,origin
lo_ordertotalprice,7975803,5244855,3467898,7480389,16136280,901571,10291840,7992196


In [36]:
column_names = ['c_custkey', 'c_name', 'c_address', 'c_city', 'c_nation', 'c_region', 'c_phone', 'c_mktsegment','']
customer_tbl = pd.read_csv('/content/ssb-dbgen/customer.tbl',sep='|\n', delimiter='|', header=None, names=column_names)
customer_tbl = customer_tbl.drop(columns='')
print(customer_tbl.info())
customer_tbl

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 8 columns):
c_custkey       60000 non-null int64
c_name          60000 non-null object
c_address       60000 non-null object
c_city          60000 non-null object
c_nation        60000 non-null object
c_region        60000 non-null object
c_phone         60000 non-null object
c_mktsegment    60000 non-null object
dtypes: int64(1), object(7)
memory usage: 3.7+ MB
None


,c_custkey,c_name,c_address,c_city,c_nation,c_region,c_phone,c_mktsegment
0,1,Customer#000000001,j5JsirBM9P,MOROCCO 2,MOROCCO,AFRICA,25-896-195-2382,BUILDING
1,2,Customer#000000002,487LW1dovn6Q4dMVym,JORDAN 8,JORDAN,MIDDLE EAST,23-689-211-2832,AUTOMOBILE
2,3,Customer#000000003,fkRGN8n,ARGENTINA5,ARGENTINA,AMERICA,11-596-827-2539,AUTOMOBILE
3,4,Customer#000000004,4u58h f,EGYPT 6,EGYPT,MIDDLE EAST,14-259-762-5556,MACHINERY
4,5,Customer#000000005,hwBtxkoBF qSW4KrI,CANADA 9,CANADA,AMERICA,13-435-393-3907,HOUSEHOLD
...,...,...,...,...,...,...,...,...
59995,59996,Customer#000059996,"xRjqF7Gijz,6Vc",UNITED KI8,UNITED KINGDOM,EUROPE,33-264-394-2297,HOUSEHOLD
59996,59997,Customer#000059997,",zvvuXOzpw",SAUDI ARA9,SAUDI ARABIA,MIDDLE EAST,30-794-278-7008,MACHINERY
59997,59998,Customer#000059998,MQDsPt21EcVF,ARGENTINA2,ARGENTINA,AMERICA,11-781-822-3298,HOUSEHOLD
59998,59999,Customer#000059999,MYAQt4c05APYy,INDIA 3,INDIA,ASIA,18-322-232-2700,BUILDING


In [38]:
algos = [ps_volume, dict_volume, pdict_volume, rle, row_huffman_volume, huffman_volume, lz78, get_uncompressed_volume]
labels = ['ps', 'dict', 'pdict', 'rle', 'row_huff', 'huff', 'lz78', 'origin', 'npz', 'gzip']
raw_value_sizes = [4 for i in range(len(algos))]
display_pair(columns_compression_volumes_df([customer_tbl.c_name,	customer_tbl.c_address,	customer_tbl.c_city, customer_tbl.c_nation, customer_tbl.c_region, customer_tbl.c_phone, customer_tbl.c_mktsegment], algos, labels, raw_value_sizes,16384))

,ps,dict,pdict,rle,row_huff,huff,lz78,origin,npz,gzip
c_name,263151,1440000,104976,65700,1889820,50362,230553,240000,198976,101171
c_address,934432,1263143,104976,75600,1709820,304206,1013424,240000,943665,119957
c_city,600000,112500,74976,65938,127421,73401,220917,240000,81641,91404
c_nation,456864,64155,52476,63035,40087,57940,197013,240000,47605,59842
c_region,388672,60810,37476,59615,18841,19300,112017,240000,24334,29687
c_phone,900000,1260000,104976,62880,1709820,149430,605670,240000,611643,66838
c_mktsegment,541440,60975,37476,60434,18979,24902,140949,240000,24382,29541


,ps,dict,pdict,rle,row_huff,huff,lz78,origin,npz,gzip
rate,2.431285,2.536657,0.307936,0.269763,3.282612,0.404489,1.500323,1.0,1.150146,0.29669


[**IMDb**](https://datasets.imdbws.com/) 

in some columns python sorting in rle method crashes because of too long string 

In [39]:
%cd /content
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!gunzip title.akas.tsv.gz

/content
--2020-01-15 16:53:54--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.226.42.104, 13.226.42.50, 13.226.42.43, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.226.42.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173108115 (165M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 165.09M  71.1MB/s    in 2.3s    

2020-01-15 16:53:57 (71.1 MB/s) - ‘title.akas.tsv.gz’ saved [173108115/173108115]



In [0]:
title_tbl = pd.read_table('/content/title.akas.tsv',low_memory=False)
title_tbl = title_tbl[:10000]

In [41]:
title_tbl

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
2,tt0000001,3,Карменсита,RU,\N,imdbDisplay,\N,0
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
4,tt0000001,5,Carmencita,\N,\N,original,\N,1
...,...,...,...,...,...,...,...,...
9995,tt0004625,1,The Spider and Her Web,US,\N,\N,\N,0
9996,tt0004626,1,The Spirit of the Conqueror,\N,\N,original,\N,1
9997,tt0004626,2,The Spirit of the Conqueror,US,\N,\N,\N,0
9998,tt0004626,3,The Spirit of the Conqueror: Or The Napoleon o...,US,\N,\N,complete title,0


In [42]:

display_pair(columns_compression_volumes_df([title_tbl.titleId, title_tbl.region], algos, labels, raw_value_sizes, 16384))

,ps,dict,pdict,rle,row_huff,huff,lz78,origin,npz,gzip
titleId,40003,79462,16532,6430,106501,8476,32796,40000,21080,8587
region,20000,10685,9996,8844,8249,7328,14241,40000,6499,7393


,ps,dict,pdict,rle,row_huff,huff,lz78,origin,npz,gzip
rate,0.750038,1.126837,0.3316,0.190925,1.434375,0.19755,0.587962,1.0,0.344738,0.19975


In [43]:
algos = [ps_volume, dict_volume, pdict_volume, row_huffman_volume, huffman_volume, lz78, get_uncompressed_volume]
labels = ['ps', 'dict', 'pdict', 'row_huff', 'huff', 'lz78', 'origin', 'npz', 'gzip']
raw_value_sizes = [4 for i in range(len(algos))]
display_pair(columns_compression_volumes_df([title_tbl.title], algos, labels, raw_value_sizes, 16384))

,ps,dict,pdict,row_huff,huff,lz78,origin,npz,gzip
title,301920,220759,17270,278034,32043,155301,40000,123964,20753


,ps,dict,pdict,row_huff,huff,lz78,origin,npz,gzip
rate,7.548,5.518975,0.43175,6.95085,0.801075,3.882525,1.0,3.0991,0.518825


In [44]:
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!gunzip title.basics.tsv.gz
title_basics_tbl = pd.read_table('/content/title.basics.tsv',low_memory=False)

--2020-01-15 16:54:49--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.226.42.104, 13.226.42.50, 13.226.42.43, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.226.42.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115142884 (110M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>] 109.81M  69.2MB/s    in 1.6s    

2020-01-15 16:54:50 (69.2 MB/s) - ‘title.basics.tsv.gz’ saved [115142884/115142884]



In [45]:
title_basics_tbl = title_basics_tbl[:10000]
title_basics_tbl

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
9995,tt0010129,movie,Fires of Faith,Fires of Faith,0,1919,\N,60,Drama
9996,tt0010130,movie,The Firing Line,The Firing Line,0,1919,\N,90,Drama
9997,tt0010131,movie,Fit to Win,Fit to Win,0,1919,\N,60,Drama
9998,tt0010132,movie,Flame of the Desert,Flame of the Desert,0,1919,\N,50,"Drama,Romance"


In [46]:
display_pair(columns_compression_volumes_df([title_basics_tbl.originalTitle, title_basics_tbl.tconst, title_basics_tbl.titleType, title_basics_tbl.genres, title_basics_tbl.primaryTitle], algos, labels, raw_value_sizes, 16384))

,ps,dict,pdict,row_huff,huff,lz78,origin,npz,gzip
originalTitle,130480,244694,17270,318410,31677,137619,40000,123873,21218
tconst,41811,150000,17270,222936,11570,35619,40000,32544,12248
titleType,50000,10054,3746,1286,899,9108,40000,2075,1911
genres,86864,19658,12496,18565,7387,32445,40000,9743,9754
primaryTitle,130480,244727,17270,318127,31789,136587,40000,122614,21290


,ps,dict,pdict,row_huff,huff,lz78,origin,npz,gzip
rate,2.198175,3.345665,0.34026,4.39662,0.41661,1.75689,1.0,1.454245,0.332105
